# NNDL实验二教程
By ChenAo 2024.6.1(六一怎么还要加班啊，我想过儿童节)<br>
实验二中，若是将代码在PyCharm中运行是得不到输出结果的，或者换种角度讲，是需要自己加``print()``的。所以在实验二的教程中我只会提供juypter版本
## 层和块
块（block）可以描述单个层、由多个层组成的组件或整个模型本身。 使用块进行抽象的一个好处是可以将一些块组合成更大的组件。有点像在CNN中卷积层和池化层是在一起使用的。那么他们两个就可以组成一个块。(比模型小，比层大)
### 自定义块
从原理角度上讲，我的块是应该要能执行反向梯度传播的，也是要能存储块中一些层的信息的，还要有块的输入输出。但是，我又不能自己手搓出各种层和配套的反向梯度传播吧。所以，最好的角度就是利用封装的思想，创造一个**块的类**，通过继承一些类，这样既不用我自己写，也可实现部分的自定义。
<br><br>
在构造自的类时候，需要继承nn.Module类，并重新实现构造函数``__init__``构造函数和``forward``这两个方法。(如果你不懂怎么继承，还有__init__是什么，请自学Python)<br>

通常情况下，我们将可学习参数的层(如全连接层、卷积层等)放在构造函数``__init__()``中，``forward``里面是层的连接关系。所谓连接关系就是给你样本你是经过什么样的流程输出的.，即给我什么样的输入，我这个层应该输出什么？forward函数返回值就是输出结果

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

X = torch.rand(2, 20)

# 在接下来代码块中，你如果发现报错，请将上面四行代码粘贴到代码块最前方,必要时你可能需要调节输入样本的大小

class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用父类Module的构造函数来执行必要的初始化。
        # 就是创建一个隐藏层和输出层
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层，输入20输出256
        self.out = nn.Linear(256, 10)  # 输出层，输入256输出10

    # 定义模型的前向传播，在这个层中我们并没有涉及到训练过程,这个函数就是单纯的给个输出值罢了
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        # 这个例子的连接关系十分简单，self.hidden(X)将样本数据给隐藏层(这里是调用类的变量，不懂的学python去)
        # 这个函数的结果又给了relu激活函数，激活后的结果又给了self.out输出层去，最后forward函数返回最终结果
        # 隐藏层->Relu->输出层
        return self.out(F.relu(self.hidden(X)))

# 在接下来代码中,我可能会省略这两步实体化
net = MLP()
net(X)
# net(X)调用其forward方法,这是PyTorch设计好的,没有为什么

tensor([[ 0.1268, -0.0797,  0.0547,  0.0217, -0.1345,  0.0852, -0.2100,  0.0048,
          0.1060, -0.0613],
        [ 0.1475, -0.0568,  0.1025,  0.1636, -0.1733,  0.0612, -0.1817,  0.0013,
          0.0766, -0.0730]], grad_fn=<AddmmBackward0>)

``net(X)``返回的是一个形状为(batch_size, output_size)的张量，其中batch_size是输入X的第一个维度大(在这个例子中是 2),output_size 是输出层``self.out``的输出特征数量在这个例子中是10<br>
所以，``net(X)``返回的是一个形状为(2, 10)的张量，它代表了模型对于输入的预测结果<br>
**这个层里面我并没有进行训练**
### 顺序块
在pytorch中``Sequential类``来实现简单的顺序连接模型<br>
这里我们不用已经有的``Sequential类``而是自己写个``MySequential类``来实现对应功能

In [8]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        # args与 enumerate(args)是python中函数输入多变量的固定写法,如果你不会,自己去学python
        # 请注意,这里面是的，self._modules 是从 nn.Module 父类继承来的
        # 它提供了一个字典_modules来存储子模块(即其他 nn.Module的实例)
        # 甚至它还是 private的
        for idx, module in enumerate(args):
            # idx：idx是enumerate返回的当前元素的索引(从0开始)。在这个例子中,它就是key值(字典的知识),用于查找每一个模块用的，用于将模块存储在self._modules字典中。
            # module是enumerate返回的当前元素，它是一个nn.Module的子类实例。它代表了模型中的一个层、激活函数或其他模块，并将在前向传播时应用于输入数据。
            # 这个例子里面就是我nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)
            # 相当于我self._modules分布存储来上面三个模块
            self._modules[str(idx)] = module

    def forward(self, X):
        # 每一个输入都是这个模块的输出,就相当于把他们以此连接起来了,实际上这个效果和MLP类是一样的
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0338, -0.2775,  0.2391,  0.2422,  0.3082,  0.1461, -0.1675, -0.0107,
          0.0950, -0.0758],
        [-0.0480, -0.1441,  0.1748,  0.0414,  0.2865,  0.1732, -0.1178,  0.0054,
         -0.0435, -0.0125]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码
这部分就是告诉你我可以在``forward``函数中执行一些运算,或者``print('陈傲牛逼')``

In [3]:
class FixedHiddenMLP(nn.Module):  
    # 定义一个名为FixedHiddenMLP的类，它继承自PyTorch的nn.Module类  
    def __init__(self):  
        super().__init__()   
        # 创建一个随机权重矩阵，大小为(20, 20)，并且不计算其梯度（即这些权重在训练过程中不会更新）  
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  
  
        # 创建一个全连接层，输入特征数为20，输出特征数也为20  
        self.linear = nn.Linear(20, 20)  
  
    def forward(self, X):  
        # 定义前向传播方法，它定义了数据X通过网络的方式  
        X = self.linear(X)  # 数据X通过全连接层self.linear  
  
        # 使用之前创建的常量权重矩阵self.rand_weight与X进行矩阵乘法，并添加一个偏置项1 
        # 然后应用ReLU激活函数  
        X = F.relu(torch.mm(X, self.rand_weight) + 1)  
  
        # 再次使用全连接层self.linear，这次是对上一步得到的X进行变换  
        # 注意：这里相当于复用了全连接层，但权重是共享的，即两次使用的都是self.linear的权重  
        X = self.linear(X)  

        print('陈傲牛逼')
        
        # 控制流：使用一个while循环来检查X的绝对值之和是否大于1  
        # 如果大于1，则将X的每个元素都除以2，直到条件不满足为止  
        while X.abs().sum() > 1:  
            X /= 2  
  
        # 返回X中所有元素的和  
        return X.sum()

net = FixedHiddenMLP()
net(X)

陈傲牛逼


tensor(-0.0333, grad_fn=<SumBackward0>)

上述代码我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量<br>
这个权重不是一个模型参数，它永远不会被反向传播更新,然后,神经网络将这个固定层的输出通过一个全连接层<br>
后面的while循环其实实际上没啥用,就是告诉你我能任意代码集成到神经网络计算的流程中<br>
我们可以混合搭配各种组合块的方法，就像下面那样

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Sequential前面已经讲过了,就是把这些模块串起来罢了
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        # 就是将nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU(), n.Linear(32, 16)一起串起来
        # 这里就是告诉你我能混合写,本来能用nn.Sequential一步到位的,非得要再搞一个
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0680, grad_fn=<SumBackward0>)

## 参数管理

### 参数权重和偏置
通过Sequential类定义模型时,我们可以通过索引来访问模型的任意层,下面的例子是访问**第三层**

In [11]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.1250, -0.1450, -0.0369,  0.3160,  0.0313,  0.0909,  0.1142,  0.0215]])), ('bias', tensor([-0.1029]))])


注意,我们最后一层有1个神经元,但是输入有8维,即有8个权重一个偏置,符合我们输出的结果<br>
实际上我的网络是有4个输入神经元;隐藏层有一层,为8个神经元,输出层有1个神经元,都是全连接<br>
值得注意的是,虽然我没有初始化权重和偏置,但是pytorch会默认为我们随机初始化的

#### 单独访问权重和偏置

下面会分别打印出`net[2].bias`的类型、`net[2].bias`本身（即包含梯度信息的参数张量）以及`net[2].bias.data`（即不包含梯度信息的、仅包含参数数据的张量）<br>
`requires_grad=True`代表这个张量需要计算梯度，即需要训练的<br>
`net[2].bias.data`通常用于在不修改其`requires_grad`属性的情况下操作张量的值（例如，在更新模型参数时）

In [12]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1029], requires_grad=True)
tensor([-0.1029])


只要执行了反向传播，并且对应的``requires_grad``属性被设置为True(运行权重更新)，``net[2].weight.grad``就不会是None<br>
这里我们没有执行反向传播,很显然不符合这个条件所以是None

In [13]:
net[2].weight.grad == None

True

#### 一次性访问所有参数
即通过递归整个树结构来提取每个子块的参数,下面我们将通过演示来比较访问第一个全连接层的参数和访问所有层<br>
很好理解,这种语句本质上就是python中的列表推导和参数解包<br>
``net[0].named_parameters()``是用于获取该模块内部所有参数的名称和对应的参数对象,就是用提取所有的参数的

In [18]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这是另一种参数访问的方法,没啥用

In [15]:
net.state_dict()['2.bias'].data

tensor([-0.1029])

#### 从嵌套块收集参数
就是层中套了个层,然后我们看看它该怎么表示

In [22]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套,4->8->4结构套四次
        net.add_module(f'block {i}', block1())
    return net

X = torch.randn(2, 4)
# 注意看,这里又接入了个4->1的,相当于1个神经元,输入是4个,输出是1个
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.2305],
        [0.2305]], grad_fn=<AddmmBackward0>)

In [23]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的,所以我们也可以像通过嵌套列表索引一样访问它们<br>
下面是访问第1个的块中第2个子块的第1层的偏置

In [35]:
rgnet[0][1][0].bias.data

tensor([-0.3105, -0.3074,  0.2569,  0.3798, -0.1806,  0.3053, -0.3865,  0.0992])

要是直接想要访问``nn.Linear(4, 1)``层偏置,就不用索引那么多层了

In [36]:
rgnet[1].bias.data

tensor([0.2305])

### 参数初始化
我们上面的权重和偏置参数都是pytorch自动随机初始化的,有的时候,我们可能希望让参数按照一定的要求进行初始化,下面就是教你怎么初始化
#### 内置初始化
这是调用内置的初始化器,下面的代码将所有权重初始化为标准差为0.01的高斯随机变量,且将偏置设置为0

In [37]:
# 定义初始化函数
def init_normal(m):
    if type(m) == nn.Linear:# 检查是不是线性层,是就初始化
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

net.apply(init_normal)# 应用我们刚才的初始化
net[0].weight.data[0], net[0].bias.data[0] # 查看初始化结果

(tensor([ 0.0038, -0.0048, -0.0032,  0.0050]), tensor(0.))

这里将所有参数初始化为给定的常数,下面是权重初始化为1,偏置初始化为0

In [11]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

对不同的块应用不同的初始化方法,其实就是提取出层然后``apply``不同的初始化函数罢了<br>
下面我们使用Xavier初始化方法初始化第1层神经网络层，然后将第3层神经网络层初始化为常量值42

In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.4554, 0.0583, 0.4283, 0.1632])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


#### 自定义初始化
如果你想要的初始化方法没有现成的函数给你调,那你就只能直接写了,下面就是直接写

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0]) # 这句打印其实没啥用,就是为了让你看清楚的
        nn.init.uniform_(m.weight, -10, 10) #  函数将权重初始化为从-10到10的均匀分布
        m.weight.data *= m.weight.data.abs() >= 5 # 对权重分布进行操作,你可以不用理解,这步就是为了满足你要求存在的

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[-5.8742,  6.6147, -7.4430, -0.0000],
        [-8.8701,  9.3737,  8.3020, -0.0000]], grad_fn=<SliceBackward0>)

当然,你随时可以考虑改变参数(什么上帝之手)

In [14]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  7.6147, -6.4430,  1.0000])

### 参数绑定
有时我们希望在多个层间共享参数,我们可以定义一个稠密层,然后使用它的参数来设置另一个层的参数<br>
其实我的理解就是共用一个层了

In [15]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8) # 这是你要共享的层
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), # 共享的层在3 5层出现了,所以3 5层参数一样的
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象,而不只是有相同的值,这就说明我动一个另一个也会跟着变(这是不是叫深拷贝来着?)
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层
此处我们构造一个没有任何参数的自定义层,继承基础层类并实现前向传播功能<br>
其实自定义层和块很像,基本上写的都是一样的,只不过块里面有很多层,而层里面只有一层,都是定义``forward``方法来指定数据通过网络时如何被处理

In [40]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__() 

    def forward(self, X):
        return X - X.mean()# 返回值是X减去X的均值,每个位置一一对应算

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5])) # 创建一个数据,扔进去让他跑以下forward

tensor([-2., -1.,  0.,  1.,  2.])

In [19]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer()) # 经典线性全连接
Y = net(torch.rand(4, 8))
Y.mean()

tensor(7.4506e-09, grad_fn=<MeanBackward0>)

### 带参数的层
就是构造函数中创建参数和权重的信息

In [44]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units)) # 权重矩阵
        self.bias = nn.Parameter(torch.randn(units,)) # 偏置矩阵
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
linear = MyLinear(5, 3) # 3个神经元,输入为5,那么偏置就有3个,权重是5*3
print(linear.weight)
print(linear.bias)

Parameter containing:
tensor([[ 0.5353,  0.3704,  1.3792],
        [-2.0872, -0.2502,  2.2236],
        [ 1.9504, -0.2793,  1.4658],
        [ 1.1301,  0.0877, -0.6527],
        [-1.4362, -0.3157,  0.8891]], requires_grad=True)
Parameter containing:
tensor([ 0.7101, -1.2621,  0.7009], requires_grad=True)


In [46]:
linear(torch.rand(2, 5))# randon第一个参数是批量大小,第二个参数是维度

tensor([[0.9857, 0.0000, 3.0985],
        [0.6625, 0.0000, 5.4142]])

In [45]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64)) # 调用得到输出结果

tensor([[0.7633],
        [0.0000]])

## 读写文件

### 读取保存张量信息(就是读取保存样本数据)
对于单个张量,我们可以直接调用load和save函数分别读写它们,这两个函数都要求我们提供一个名称,save要求将要保存的变量作为输入

In [23]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4) # 生成张量
torch.save(x, 'x-file') # 将张量保存为x-file文件

x2 = torch.load('x-file') # 读取x-file文件中张量,放到x2中
x2

tensor([0, 1, 2, 3])

你要是有两个张量呢,也可以

In [24]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

你甚至可以写入或读取从字符串映射到张量的字典

In [25]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

### 加载和保存模型参数
就是将模型里面所有的参数(权重和偏置)保存起来,毕竟你要是像上面那样挨个保存得累死了

In [26]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

torch.save(net.state_dict(), 'mlp.params')# 保存参数到mlp.params文件中

clone = MLP()
clone.load_state_dict(torch.load('mlp.params')) # 读取mlp.params文件中参数到clone模型中
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [27]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

注意,两个模型只是参数相同罢了,你要是修改一个参数另一个模型参数不会改变的